<a href="https://colab.research.google.com/github/SarabRajesh/Explainable-AI-Pract-B-44/blob/main/Weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import pandas as pd

# Step 1: Unzip the file
zip_path = '/content/weather.csv.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Extracts in the same directory

# Step 2: Find the CSV file (assuming it's called 'weather.csv')
# Corrected filename based on available files
csv_path = '/content/weatherAUS.csv'

# Step 3: Load the CSV file using pandas
df = pd.read_csv(csv_path)

# Step 4: Display some basic information
print("✅ File loaded successfully!\n")
print("📊 First 5 rows of the dataset:")
print(df.head())

print("\n🧾 DataFrame Info:")
print(df.info())

✅ File loaded successfully!

📊 First 5 rows of the dataset:
         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am 

In [ ]:
import pandas as pd

# Load your data (adjust path if needed)
df = pd.read_csv('/content/weatherAUS.csv')

# OPTIONAL: Check the column names
print("🧾 Columns in dataset:")
print(df.columns)

# Define your target column
target_column = 'RainTomorrow'  # Replace this with your actual target column name

# Separate features and target
X = df.drop(columns=[target_column])  # All columns except the target
y = df[target_column]                 # Only the target column

print("\n✅ Features (X) shape:", X.shape)
print("🎯 Target (y) shape:", y.shape)

🧾 Columns in dataset:
Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

✅ Features (X) shape: (145460, 22)
🎯 Target (y) shape: (145460,)


In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets (you can adjust the test_size)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Data split successfully!")
print("🏋️ Training data shape:", X_train.shape)
print("🧪 Validation data shape:", X_val.shape)
print("🎯 Training target shape:", y_train.shape)
print("🎯 Validation target shape:", y_val.shape)

✅ Data split successfully!
🏋️ Training data shape: (116368, 22)
🧪 Validation data shape: (29092, 22)
🎯 Training target shape: (116368,)
🎯 Validation target shape: (29092,)


CNN

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the data (assuming the previous steps loaded it into df, X, and y)
# If not, uncomment and run the following lines:
# df = pd.read_csv('/content/weatherAUS.csv')
# target_column = 'RainTomorrow'
# X = df.drop(columns=[target_column])
# y = df[target_column]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# --- Preprocessing Steps ---

# Identify categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns

# Create preprocessing pipelines for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Impute missing numerical values
    ('scaler', StandardScaler()) # Scale numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Impute missing categorical values
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # One-hot encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply preprocessing to the training and validation data
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# Preprocess target variable (RainTomorrow: Yes/No to 1/0)
y_train_processed = y_train.apply(lambda x: 1 if x == 'Yes' else (0 if x == 'No' else -1)) # Map Yes to 1, No to 0, and others to -1
y_val_processed = y_val.apply(lambda x: 1 if x == 'Yes' else (0 if x == 'No' else -1))

# Remove rows where target variable is -1 (due to NaN in original y)
valid_train_indices = y_train_processed != -1
X_train_processed = X_train_processed[valid_train_indices.to_numpy()] # Convert boolean Series to numpy array
y_train_processed = y_train_processed[valid_train_indices]

valid_val_indices = y_val_processed != -1
X_val_processed = X_val_processed[valid_val_indices.to_numpy()] # Convert boolean Series to numpy array
y_val_processed = y_val_processed[valid_val_indices]


# --- Model Definition and Training ---

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_processed.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Use binary_crossentropy for binary classification
model.fit(X_train_processed, y_train_processed, epochs=10, validation_data=(X_val_processed, y_val_processed)) # Reduced epochs for faster execution

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - accuracy: 0.8462 - loss: 0.3571 - val_accuracy: 0.8644 - val_loss: 0.3117
Epoch 2/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.8804 - loss: 0.2786 - val_accuracy: 0.8685 - val_loss: 0.3016
Epoch 3/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.8986 - loss: 0.2380 - val_accuracy: 0.8707 - val_loss: 0.3012
Epoch 4/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - accuracy: 0.9148 - loss: 0.1989 - val_accuracy: 0.8700 - val_loss: 0.3238
Epoch 5/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.9335 - loss: 0.1591 - val_accuracy: 0.8679 - val_loss: 0.3543
Epoch 6/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.9486 - loss: 0.1258 - val_accuracy: 0.8642 - val_loss: 0.3899
Epoch 7/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.9611 - loss: 0.0984 - val_accuracy: 0.8649 - val_loss: 0.4685
Epoch 8/10
3556/3556 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.9727 - loss: 0

rnn

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import numpy as np

# Convert sparse matrices to dense NumPy arrays before reshaping
X_train_processed_dense = X_train_processed.todense()
X_val_processed_dense = X_val_processed.todense()

# Reshape the input data for LSTM (assuming each feature is a timestep)
# The shape should be (samples, timesteps, features)
# Here, we treat each feature as a single timestep, so features = 1
timesteps = X_train_processed_dense.shape[1]
features = 1
X_train_reshaped = np.reshape(X_train_processed_dense, (X_train_processed_dense.shape[0], timesteps, features))
X_val_reshaped = np.reshape(X_val_processed_dense, (X_val_processed_dense.shape[0], timesteps, features))


model = Sequential([
    Input(shape=(timesteps, features)), # Explicitly define Input layer
    LSTM(64, activation='tanh'),
    Dense(1, activation='sigmoid') # Changed to sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Changed to binary_crossentropy for binary classification
model.fit(X_train_reshaped, y_train_processed, epochs=10, validation_data=(X_val_reshaped, y_val_processed)) # Reduced epochs for faster execution

Epoch 1/10
2804/3556 ━━━━━━━━━━━━━━━━━━━━ 16:03 1s/step - accuracy: 0.7764 - loss: 0.5027

shap

In [ ]:
import shap
import xgboost as xgb

# Train an XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Explain the model using SHAP
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test)

# Plot global feature importance
shap.summary_plot(shap_values, X_test)

# Plot for one prediction
shap.plots.waterfall(shap_values[0])


lime

In [ ]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X.columns,
    mode='regression'
)

# Explain one instance
i = 5  # Index of the sample
exp = explainer.explain_instance(X_test.iloc[i].values, xgb_model.predict)

# Show explanation
exp.show_in_notebook(show_table=True)


pdp

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Load dataset
data = pd.read_csv("/content/weatherAUS.csv")

# Drop rows with missing target
data = data.dropna(subset=['RainTomorrow'])

# Select features (you can adjust this list)
features = ['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity9am', 'Humidity3pm', 'WindSpeed9am']
X = data[features]
y = data['RainTomorrow']

# Encode target
le = LabelEncoder()
y = le.fit_transform(y)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Partial Dependence Plot for selected features
fig, ax = plt.subplots(figsize=(12, 8))
PartialDependenceDisplay.from_estimator(model, X_test, features=['MaxTemp', 'Humidity3pm'], ax=ax)
plt.show()
